In [1]:
import utils
from conversation import ConversationConfig, Analyzer

In [2]:
utils.load_secrets()
output_dir = utils.output_dir()

In [3]:
role0_system_prompt = (
    "You are in a conversation with a friend you have a slight crush on"
)
role1_system_prompt = (
    "You are in a conversation with a friend you need to ask for money"
)
role0_start_message = "How are you doing?"

In [4]:
config = ConversationConfig(
    role0_system_prompt=role0_system_prompt,
    role1_system_prompt=role1_system_prompt,
    role0_start_message=role0_start_message,
    role0_name="Alice",
    role1_name="Bob",
)
conversation = config.create_conversation()

In [5]:
conversation = config.create_conversation()

In [ ]:
conversation.advance(5)
conversation.print_messages()

In [7]:
conversation.save_to_file(output_dir / "conversation.json")

In [8]:
analyzer = Analyzer(
    system_prompt="You are a Freudian psychotherapist",
    instruction="Analyze the conversation and give it a score from 0 to 1 for how unhinged it is",
)

In [ ]:
print(analyzer.analyze(conversation))

In [10]:
scores = [analyzer.analyze(conversation).score for _ in range(10)]

In [ ]:
scores